# PyDough TPC-H

This notebook provides PyDough translations for 20 out of the 22 TPC-H benchmark queries. For each query we will reproduce the business question, the corresponding SQL query, and finally a valid PyDough implementation. The underlying schema of this data matches this example image from [TPC Benchmark H Standard Specification](https://www.tpc.org/tpc_documents_current_versions/pdf/tpc-h_v2.17.1.pdf).

![TPC-H schema from the Specification Document as of December 12, 2024](../images/tpc_h_schema.png)

In several places we have update the names in our metadata to be more human readable. It should also be noted that the TPC-H benchmark does not necessarily constitute the "simplest" way to express the SQL for each of these statements, but this comparison is still useful for understanding how to construct PyDough statements.

In [1]:
%load_ext pydough_jupyter_extensions

In [2]:
import pydough
import datetime
import pandas as pd

In [3]:
# Setup demo metadata
pydough.active_session.load_metadata_graph("../metadata/tpch_demo_graph.json", "TPCH");
pydough.active_session.connect_database("sqlite", database="../tpch.db");

## Query 1

This query seeks to answer the question 

Here is the corresponding SQL:

```SQL
select
	l_returnflag,
	l_linestatus,
	sum(l_quantity) as sum_qty,
	sum(l_extendedprice) as sum_base_price,
	sum(l_extendedprice * (1 - l_discount)) as sum_disc_price,
	sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) as sum_charge,
	avg(l_quantity) as avg_qty,
	avg(l_extendedprice) as avg_price,
	avg(l_discount) as avg_disc,
	count(*) as count_order
from
	lineitem
where
	l_shipdate <= date '1998-12-01' - interval '90' day
group by
	l_returnflag,
	l_linestatus
order by
	l_returnflag,
	l_linestatus
LIMIT 1;
```

In [10]:
%%pydough

selected_lines = lines.WHERE((ship_date <= datetime.date(1998, 9, 2)))
output = PARTITION(selected_lines, name="l", by=(return_flag, status))(
    L_RETURNFLAG=return_flag,
    L_LINESTATUS=status,
    SUM_QTY=SUM(l.quantity),
    SUM_BASE_PRICE=SUM(l.extended_price),
    SUM_DISC_PRICE=SUM(l.extended_price * (1 - l.discount)),
    SUM_CHARGE=SUM(l.extended_price * (1 - l.discount) * (1 + l.tax)),
    AVG_QTY=AVG(l.quantity),
    AVG_PRICE=AVG(l.extended_price),
    AVG_DISC=AVG(l.discount),
    COUNT_ORDER=COUNT(l),
).ORDER_BY(L_RETURNFLAG.ASC(), L_LINESTATUS.ASC())
pydough.to_df(output)

,L_RETURNFLAG,L_LINESTATUS,SUM_QTY,SUM_BASE_PRICE,SUM_DISC_PRICE,SUM_CHARGE,AVG_QTY,AVG_PRICE,AVG_DISC,COUNT_ORDER
0,A,F,37734107,5.658655e+10,5.375826e+10,5.590907e+10,25.522006,38273.129735,0.049985,1478493
1,N,F,991417,1.487505e+09,1.413082e+09,1.469649e+09,25.516472,38284.467761,0.050093,38854
2,N,O,76633518,1.149352e+11,1.091896e+11,1.135610e+11,25.502020,38248.015609,0.050000,3004998
3,R,F,37719753,5.656804e+10,5.374129e+10,5.588962e+10,25.505794,38250.854626,0.050009,1478870


## Query 2

```SQL
SELECT
    S_ACCTBAL,
    S_NAME,
    N_NAME,
    P_PARTKEY,
    P_MFGR,
    S_ADDRESS,
    S_PHONE,
    S_COMMENT
FROM
    PART,
    SUPPLIER,
    PARTSUPP,
    NATION,
    REGION
WHERE
    P_PARTKEY = PS_PARTKEY
    AND S_SUPPKEY = PS_SUPPKEY
    AND P_SIZE = 15
    AND P_TYPE LIKE '%BRASS'
    AND S_NATIONKEY = N_NATIONKEY
    AND N_REGIONKEY = R_REGIONKEY
    AND R_NAME = 'EUROPE'
    AND PS_SUPPLYCOST = (
        SELECT MIN(PS_SUPPLYCOST)
        FROM PARTSUPP, SUPPLIER, NATION, REGION
        WHERE P_PARTKEY = PS_PARTKEY
          AND S_SUPPKEY = PS_SUPPKEY
          AND S_NATIONKEY = N_NATIONKEY
          AND N_REGIONKEY = R_REGIONKEY
          AND R_NAME = 'EUROPE'
    )
ORDER BY
    S_ACCTBAL DESC,
    N_NAME,
    S_NAME,
    P_PARTKEY
LIMIT 100;
```

In [12]:
%%pydough

selected_parts = (
    nations.WHERE(region.name == "EUROPE")
    .suppliers.supply_records.part(
        s_acctbal=BACK(2).account_balance,
        s_name=BACK(2).name,
        n_name=BACK(3).name,
        s_address=BACK(2).address,
        s_phone=BACK(2).phone,
        s_comment=BACK(2).comment,
        supplycost=BACK(1).supplycost,
    )
    .WHERE(ENDSWITH(part_type, "BRASS") & (size == 15))
)
output = PARTITION(selected_parts, name="p", by=key)(
    best_cost=MIN(p.supplycost)
).p.WHERE(
    (supplycost == BACK(1).best_cost)
    & ENDSWITH(part_type, "BRASS")
    & (size == 15)
)(
    S_ACCTBAL=s_acctbal,
    S_NAME=s_name,
    N_NAME=n_name,
    P_PARTKEY=key,
    P_MFGR=manufacturer,
    S_ADDRESS=s_address,
    S_PHONE=s_phone,
    S_COMMENT=s_comment,
).TOP_K(
    100,
    by=(S_ACCTBAL.DESC(), N_NAME.ASC(), S_NAME.ASC(), P_PARTKEY.ASC()),
)
pydough.to_df(output)

,S_ACCTBAL,S_NAME,N_NAME,P_PARTKEY,P_MFGR,S_ADDRESS,S_PHONE,S_COMMENT
0,9938.53,Supplier#000005359,UNITED KINGDOM,185358,Manufacturer#4,"QKuHYh,vZGiwu2FWEJoLDx04",33-429-790-6131,uriously regular requests hag
1,9937.84,Supplier#000005969,ROMANIA,108438,Manufacturer#1,"ANDENSOSmk,miq23Xfb5RWt6dvUcvt6Qa",29-520-692-3537,efully express instructions. regular requests ...
2,9936.22,Supplier#000005250,UNITED KINGDOM,249,Manufacturer#4,B3rqp0xbSEim4Mpy2RH J,33-320-228-2957,etect about the furiously final accounts. slyl...
3,9923.77,Supplier#000002324,GERMANY,29821,Manufacturer#4,y3OD9UywSTOk,17-779-299-1839,ackages boost blithely. blithely regular depos...
4,9871.22,Supplier#000006373,GERMANY,43868,Manufacturer#5,J8fcXWsTqM,17-813-485-8637,etect blithely bold asymptotes. fluffily ironi...
...,...,...,...,...,...,...,...,...
95,7887.08,Supplier#000009792,GERMANY,164759,Manufacturer#3,Y28ITVeYriT3kIGdV2K8fSZ V2UqT5H1Otz,17-988-938-4296,ckly around the carefully fluffy theodolites. ...
96,7871.50,Supplier#000007206,RUSSIA,104695,Manufacturer#1,3w fNCnrVmvJjE95sgWZzvW,32-432-452-7731,ironic requests. furiously final theodolites c...
97,7852.45,Supplier#000005864,RUSSIA,8363,Manufacturer#4,"WCNfBPZeSXh3h,c",32-454-883-3821,usly unusual pinto beans. brave ideas sleep ca...
98,7850.66,Supplier#000001518,UNITED KINGDOM,86501,Manufacturer#1,ONda3YJiHKJOC,33-730-383-3892,ifts haggle fluffily pending pai


## Query 3

```SQL
SELECT
    L_ORDERKEY,
    SUM(L_EXTENDEDPRICE * (1 - L_DISCOUNT)) AS REVENUE,
    O_ORDERDATE,
    O_SHIPPRIORITY
FROM
    CUSTOMER,
    ORDERS,
    LINEITEM
WHERE
    C_MKTSEGMENT = 'BUILDING'
    AND C_CUSTKEY = O_CUSTKEY
    AND L_ORDERKEY = O_ORDERKEY
    AND O_ORDERDATE < DATE '1995-03-15'
    AND L_SHIPDATE > DATE '1995-03-15'
GROUP BY
    L_ORDERKEY,
    O_ORDERDATE,
    O_SHIPPRIORITY
ORDER BY
    REVENUE DESC,
    O_ORDERDATE
LIMIT 10;
```

In [14]:
%%pydough

selected_lines = orders.WHERE(
    (customer.mktsegment == "BUILDING") & (order_date < datetime.date(1995, 3, 15))
).lines.WHERE(ship_date > datetime.date(1995, 3, 15))(
    BACK(1).order_date,
    BACK(1).ship_priority,
)
output = PARTITION(
    selected_lines, name="l", by=(order_key, order_date, ship_priority)
)(
    L_ORDERKEY=order_key,
    REVENUE=SUM(l.extended_price * (1 - l.discount)),
    O_ORDERDATE=order_date,
    O_SHIPPRIORITY=ship_priority,
).TOP_K(10, by=(REVENUE.DESC(), O_ORDERDATE.ASC(), L_ORDERKEY.ASC()))
pydough.to_df(output)

,L_ORDERKEY,REVENUE,O_ORDERDATE,O_SHIPPRIORITY
0,2456423,406181.0111,1995-03-05,0
1,3459808,405838.6989,1995-03-04,0
2,492164,390324.0610,1995-02-19,0
3,1188320,384537.9359,1995-03-09,0
4,2435712,378673.0558,1995-02-26,0
5,4878020,378376.7952,1995-03-12,0
6,5521732,375153.9215,1995-03-13,0
7,2628192,373133.3094,1995-02-22,0
8,993600,371407.4595,1995-03-05,0
9,2300070,367371.1452,1995-03-13,0


## Query 4

```SQL
SELECT
    O_ORDERPRIORITY,
    COUNT(*) AS ORDER_COUNT
FROM
    ORDERS
WHERE
    O_ORDERDATE >= DATE '1993-07-01'
    AND O_ORDERDATE < DATE '1993-10-01'
    AND EXISTS (
        SELECT 1
        FROM LINEITEM
        WHERE L_ORDERKEY = O_ORDERKEY
          AND L_COMMITDATE < L_RECEIPTDATE
    )
GROUP BY
    O_ORDERPRIORITY
ORDER BY
    O_ORDERPRIORITY;
```

In [15]:
%%pydough

selected_lines = lines.WHERE(commit_date < receipt_date)
selected_orders = orders.WHERE(
    (order_date >= datetime.date(1993, 7, 1))
    & (order_date < datetime.date(1993, 10, 1))
    & HAS(selected_lines)
)
output = PARTITION(selected_orders, name="o", by=order_priority)(
    O_ORDERPRIORITY=order_priority,
    ORDER_COUNT=COUNT(o),
).ORDER_BY(O_ORDERPRIORITY.ASC())
pydough.to_df(output)

,O_ORDERPRIORITY,ORDER_COUNT
0,1-URGENT,10594
1,2-HIGH,10476
2,3-MEDIUM,10410
3,4-NOT SPECIFIED,10556
4,5-LOW,10487


## Query 6

```SQL
```

In [17]:
%%pydough

selected_lines = lines.WHERE(
    (ship_date >= datetime.date(1994, 1, 1))
    & (ship_date < datetime.date(1995, 1, 1))
    & (0.05 <= discount)
    & (discount <= 0.07)
    & (quantity < 24)
)(amt=extended_price * discount)
output = TPCH(REVENUE=SUM(selected_lines.amt))
pydough.to_df(output)

,REVENUE
0,1.231411e+08


## Query 7

```SQL
```

In [18]:
%%pydough

line_info = lines(
    supp_nation=supplier.nation.name,
    cust_nation=order.customer.nation.name,
    l_year=YEAR(ship_date),
    volume=extended_price * (1 - discount),
).WHERE(
    (ship_date >= datetime.date(1995, 1, 1))
    & (ship_date <= datetime.date(1996, 12, 31))
    & (
        ((supp_nation == "FRANCE") & (cust_nation == "GERMANY"))
        | ((supp_nation == "GERMANY") & (cust_nation == "FRANCE"))
    )
)

output = PARTITION(line_info, name="l", by=(supp_nation, cust_nation, l_year))(
    SUPP_NATION=supp_nation,
    CUST_NATION=cust_nation,
    L_YEAR=l_year,
    REVENUE=SUM(l.volume),
).ORDER_BY(
    SUPP_NATION.ASC(),
    CUST_NATION.ASC(),
    L_YEAR.ASC(),
)
pydough.to_df(output)

,SUPP_NATION,CUST_NATION,L_YEAR,REVENUE
0,FRANCE,GERMANY,1995,5.463973e+07
1,FRANCE,GERMANY,1996,5.463308e+07
2,GERMANY,FRANCE,1995,5.253175e+07
3,GERMANY,FRANCE,1996,5.252055e+07


## Query 8

```SQL
```